In [120]:
import pandas as pd
import numpy as np
import nltk 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from gensim import corpora, models, similarities, matutils
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from scipy.sparse import csc_matrix
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.lancaster import LancasterStemmer



In [2]:
from pymongo import MongoClient
from pprint import pprint

client = MongoClient()

db = client.strain_project

db.strains

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'strain_project'), 'strains')

In [3]:
strain_initial = pd.DataFrame(db.strains.find())

In [4]:
strain_initial.head()

,_id,username,url,review,strain_name
0,5dc33348e1127a705e2ce279,luvlivinlife,https://www.leafly.com/sativa/gorkle/reviews/1...,“40% head 60% body. Nice flavor 20 or 30 min b...,gorkle
1,5dc33348e1127a705e2ce27a,luvlivinlife,https://www.leafly.com/sativa/gorkle/reviews/1...,"“Love this strain. Happy, Calm, Relaxed but no...",gorkle
2,5dc33349e1127a705e2ce27b,wespeterson24,https://www.leafly.com/hybrid/longbottom-leaf/...,“Are you looking to get shire-smacked in the f...,longbottom-leaf
3,5dc33349e1127a705e2ce27c,mtn420man,https://www.leafly.com/hybrid/longbottom-leaf/...,"“Such a smooth smoke. Instant head high, very ...",longbottom-leaf
4,5dc33349e1127a705e2ce27d,rjjia,https://www.leafly.com/hybrid/longbottom-leaf/...,“I now truly understand the beauty of middle e...,longbottom-leaf


In [5]:
strain_initial.shape

(898280, 5)

In [6]:
strain_initial=strain_initial.drop_duplicates(subset="review")

In [7]:
strain_initial.reset_index()

,index,_id,username,url,review,strain_name
0,0,5dc33348e1127a705e2ce279,luvlivinlife,https://www.leafly.com/sativa/gorkle/reviews/1...,“40% head 60% body. Nice flavor 20 or 30 min b...,gorkle
1,1,5dc33348e1127a705e2ce27a,luvlivinlife,https://www.leafly.com/sativa/gorkle/reviews/1...,"“Love this strain. Happy, Calm, Relaxed but no...",gorkle
2,2,5dc33349e1127a705e2ce27b,wespeterson24,https://www.leafly.com/hybrid/longbottom-leaf/...,“Are you looking to get shire-smacked in the f...,longbottom-leaf
3,3,5dc33349e1127a705e2ce27c,mtn420man,https://www.leafly.com/hybrid/longbottom-leaf/...,"“Such a smooth smoke. Instant head high, very ...",longbottom-leaf
4,4,5dc33349e1127a705e2ce27d,rjjia,https://www.leafly.com/hybrid/longbottom-leaf/...,“I now truly understand the beauty of middle e...,longbottom-leaf
...,...,...,...,...,...,...
165926,224565,5dc3335ee1127a705e304fae,MizzGizzi,https://www.leafly.com/hybrid/blue-magoo/revie...,“I really liked this strain for my day time. I...,blue-magoo
165927,224566,5dc3335ee1127a705e304faf,RecWreck,https://www.leafly.com/hybrid/blue-magoo/revie...,“Made me giggly with a very noticeable headkic...,blue-magoo
165928,224567,5dc3335ee1127a705e304fb0,samboogie,https://www.leafly.com/hybrid/blue-magoo/revie...,“just hit the bammer with this bud wow i feel ...,blue-magoo
165929,224568,5dc3335ee1127a705e304fb1,dapperErl,https://www.leafly.com/hybrid/blue-magoo/revie...,“The Blue Magoo from Forbidden Farms is FIRE!!...,blue-magoo


In [8]:
strain_initial.shape

(165931, 5)

In [9]:
strain_1 = strain_initial.copy()

In [10]:
strain_1.isna().any()

_id            False
username       False
url             True
review          True
strain_name    False
dtype: bool

In [11]:
strain_1=strain_1.dropna(subset=['review','url'])

In [12]:
strain_1.isna().any()

_id            False
username       False
url            False
review         False
strain_name    False
dtype: bool

In [13]:
strain_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165928 entries, 0 to 224569
Data columns (total 5 columns):
_id            165928 non-null object
username       165928 non-null object
url            165928 non-null object
review         165928 non-null object
strain_name    165928 non-null object
dtypes: object(5)
memory usage: 7.6+ MB


In [14]:
strain_1['strain_name'].value_counts()

sour-diesel                  3166
gsc                          3108
green-crack                  2764
og-kush                      2354
original-glue                2081
                             ... 
misty-morning                   1
perfect-purple                  1
ghostbreath                     1
grandma-andersons-cookies       1
truffula-tree                   1
Name: strain_name, Length: 2543, dtype: int64

In [15]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [16]:
strain_count=pd.DataFrame(strain_1['strain_name'].value_counts())

In [17]:
strain_count.head(50)

,strain_name
sour-diesel,3166
gsc,3108
green-crack,2764
og-kush,2354
original-glue,2081
granddaddy-purple,2045
jack-herer,1836
white-widow,1710
bubba-kush,1634
pineapple-express,1520


In [18]:
import re # Regular expression library
import string

In [19]:
strain_1['clean_review']=strain_1['review']
strain_1.head(3)

,_id,username,url,review,strain_name,clean_review
0,5dc33348e1127a705e2ce279,luvlivinlife,https://www.leafly.com/sativa/gorkle/reviews/1...,“40% head 60% body. Nice flavor 20 or 30 min b...,gorkle,“40% head 60% body. Nice flavor 20 or 30 min b...
1,5dc33348e1127a705e2ce27a,luvlivinlife,https://www.leafly.com/sativa/gorkle/reviews/1...,"“Love this strain. Happy, Calm, Relaxed but no...",gorkle,"“Love this strain. Happy, Calm, Relaxed but no..."
2,5dc33349e1127a705e2ce27b,wespeterson24,https://www.leafly.com/hybrid/longbottom-leaf/...,“Are you looking to get shire-smacked in the f...,longbottom-leaf,“Are you looking to get shire-smacked in the f...


In [20]:
def clean_text (text):
    new_text= re.sub('[%s]' % re.escape(string.punctuation), ' ',text) 
    new_text=re.sub('\w*\d\w*', ' ',new_text)
    new_text=new_text.lower()
    new_text=re.sub(r"[^a-zA-Z0-9]+", ' ',new_text)
    
    return new_text

In [21]:
strain_1['clean_review'] = strain_1['review'].apply(lambda x: clean_text(x))

In [22]:
strain_1.head(5)

,_id,username,url,review,strain_name,clean_review
0,5dc33348e1127a705e2ce279,luvlivinlife,https://www.leafly.com/sativa/gorkle/reviews/1...,“40% head 60% body. Nice flavor 20 or 30 min b...,gorkle,head body nice flavor or min before full affe...
1,5dc33348e1127a705e2ce27a,luvlivinlife,https://www.leafly.com/sativa/gorkle/reviews/1...,"“Love this strain. Happy, Calm, Relaxed but no...",gorkle,love this strain happy calm relaxed but not g...
2,5dc33349e1127a705e2ce27b,wespeterson24,https://www.leafly.com/hybrid/longbottom-leaf/...,“Are you looking to get shire-smacked in the f...,longbottom-leaf,are you looking to get shire smacked in the f...
3,5dc33349e1127a705e2ce27c,mtn420man,https://www.leafly.com/hybrid/longbottom-leaf/...,"“Such a smooth smoke. Instant head high, very ...",longbottom-leaf,such a smooth smoke instant head high very me...
4,5dc33349e1127a705e2ce27d,rjjia,https://www.leafly.com/hybrid/longbottom-leaf/...,“I now truly understand the beauty of middle e...,longbottom-leaf,i now truly understand the beauty of middle e...


In [23]:
strain_1.loc[strain_1["review"].str.contains("fire")]

,_id,username,url,review,strain_name,clean_review
169,5dc33349e1127a705e2ce322,Chill_Panda,https://www.leafly.com/indica/flaming-cookies/...,“This is a really great strain fire go which i...,flaming-cookies,this is a really great strain fire go which i...
178,5dc33349e1127a705e2ce32b,jakeprnhs,https://www.leafly.com/indica/flaming-cookies/...,“So what I smoked was called fire scouts. I be...,flaming-cookies,so what i smoked was called fire scouts i bel...
453,5dc33349e1127a705e2ce43e,MyJokerHandStayLIT,https://www.leafly.com/hybrid/slimer-og/review...,“Slymer or (Slimer) from Moxie! I’ve literally...,slimer-og,slymer or slimer from moxie i ve literally be...
683,5dc33349e1127a705e2ce524,OldLion,https://www.leafly.com/indica/afghani/reviews/...,“I was in the US Army for 3 years until I deci...,afghani,i was in the us army for years until i decide...
988,5dc33349e1127a705e2ce655,ducky518,https://www.leafly.com/indica/afghani/reviews/...,"“My favorite strain, this shit is fire af🔥👌”",afghani,my favorite strain this shit is fire af
...,...,...,...,...,...,...
224187,5dc3335ee1127a705e304e34,moemiami,https://www.leafly.com/hybrid/irene-kush/revie...,“Great for daytime use. My synapses fire so b...,irene-kush,great for daytime use my synapses fire so bea...
224334,5dc3335ee1127a705e304ec7,Statusdropout,https://www.leafly.com/hybrid/blue-magoo/revie...,“I enjoyed the white magoo. It was some good f...,blue-magoo,i enjoyed the white magoo it was some good fire
224455,5dc3335ee1127a705e304f40,McCaul,https://www.leafly.com/hybrid/blue-magoo/revie...,“this shit is fire as fuck can't keep it!”,blue-magoo,this shit is fire as fuck can t keep it
224556,5dc3335ee1127a705e304fa5,tiffpankey,https://www.leafly.com/hybrid/blue-magoo/revie...,"“Acquired a premium harvest of this strain, 25...",blue-magoo,acquired a premium harvest of this strain thc...


In [24]:
strain_1.loc[strain_1["username"]=='ducky518']

,_id,username,url,review,strain_name,clean_review
988,5dc33349e1127a705e2ce655,ducky518,https://www.leafly.com/indica/afghani/reviews/...,"“My favorite strain, this shit is fire af🔥👌”",afghani,my favorite strain this shit is fire af


In [25]:
strain_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165928 entries, 0 to 224569
Data columns (total 6 columns):
_id             165928 non-null object
username        165928 non-null object
url             165928 non-null object
review          165928 non-null object
strain_name     165928 non-null object
clean_review    165928 non-null object
dtypes: object(6)
memory usage: 8.9+ MB


## LSA

In [98]:
stop_words = text.ENGLISH_STOP_WORDS.union(["little","great","love","happy","effects","time","feel","feeling","definitely","strains","pretty","good","indica","sativa","high","strain","like","smoke","buds","bud","just","smoking","smoke","really","pretty","makes","ve","got"])

In [99]:
vectorizer = CountVectorizer(stop_words=stop_words,max_df=.7,min_df=.045,ngram_range=(1, 2))
doc_word = vectorizer.fit_transform(strain_1['clean_review'])
doc_word

<165928x21 sparse matrix of type '<class 'numpy.int64'>'
	with 263382 stored elements in Compressed Sparse Row format>

In [84]:
lsa = TruncatedSVD(5)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_

array([0.12129977, 0.09844394, 0.08067001, 0.07132624, 0.05538288])

In [85]:
topic_word = pd.DataFrame(lsa.components_.round(4),
             index = ["component_1","component_2","component_3","component_4","component_5"],
             columns = vectorizer.get_feature_names())
topic_word

,amazing,anxiety,best,body,day,favorite,flavor,great,head,little,nice,pain,recommend,relaxed,relaxing,sleep,smell,smoked,smooth,strong,super,sweet,taste
component_1,0.0886,0.1095,0.0815,0.1922,0.1342,0.0777,0.0808,0.7368,0.1302,0.0773,0.2703,0.2347,0.0669,0.0823,0.1093,0.0801,0.1947,0.0762,0.0811,0.1101,0.0661,0.1070,0.3228
component_2,0.0629,0.0310,0.0572,0.1682,0.0220,0.0360,0.0507,-0.6340,0.1135,0.0654,0.5215,0.0977,0.0364,0.0571,0.0629,0.0191,0.2454,0.0554,0.0778,0.1157,0.0430,0.1340,0.3835
component_3,-0.0064,0.1682,0.0401,0.2490,0.0826,0.0233,-0.0015,-0.1165,0.1318,0.0325,0.1132,0.6924,0.0476,0.0660,0.0581,0.0974,-0.2725,0.0274,-0.0223,-0.0069,0.0075,-0.0904,-0.5228
component_4,-0.0914,-0.0828,-0.0899,0.0556,-0.0012,-0.0524,0.0246,0.1600,0.0419,-0.0063,0.7072,-0.4762,-0.0314,-0.0002,0.0402,-0.0390,-0.1828,-0.0270,0.0043,-0.0633,-0.0099,-0.0763,-0.4097
component_5,0.0923,0.0415,0.1095,0.6660,0.0410,0.0679,0.0317,-0.0647,0.3731,0.0485,-0.3465,-0.4043,0.0313,0.0974,0.0826,0.0213,0.0352,0.1199,0.0081,0.1924,0.0728,0.0353,-0.1435


In [86]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [87]:
display_topics(lsa, vectorizer.get_feature_names(), 10)


Topic  0
great, taste, nice, pain, smell, body, day, head, strong, anxiety

Topic  1
nice, taste, smell, body, sweet, strong, head, pain, smooth, little

Topic  2
pain, body, anxiety, head, nice, sleep, day, relaxed, relaxing, recommend

Topic  3
nice, great, body, head, relaxing, flavor, smooth, relaxed, day, little

Topic  4
body, head, strong, smoked, best, relaxed, amazing, relaxing, super, favorite


In [32]:
Vt = pd.DataFrame(doc_topic.round(4),
             index = strain_1['clean_review'],
             columns = ["component_1","component_2","component_3","component_4"])
Vt

,component_1,component_2,component_3,component_4
clean_review,,,,
head body nice flavor or min before full affect clear headed but very relaxed reduced pain productive clear thinking very nice overall feeling not stuck to couch but body fully relaxed after a long week this is new favorite i am ready for bed hrs after a good vape taste with a little addition an hour ago and so is it good nite,2.0888,2.0756,0.6014,1.1542
love this strain happy calm relaxed but not glued down i can really settle in to a show or game and enjoy either i think this will be a favorite for a long time,0.5601,0.3440,0.4029,-0.4102
are you looking to get shire smacked in the face with a peppy yet tranquil bud if so look no further than this strain with colorful fluffy nugs and beautiful aroma this shit really is the precious smokes great and the flavor is stellar the high gives you a relaxed stroking your gandalf beard in the shire feel,0.9908,-0.4586,0.1758,0.0957
such a smooth smoke instant head high very meoow on the lungs but very potent for pain such a smoke i picked some up at the verdes foundation in abq nm what amazing pre rolls of this im looking forward to the next time they have this in,0.7034,0.4462,0.8004,-0.3991
i now truly understand the beauty of middle earth after smoking this earth blended gift from the fairies i no longer fear the nameless enemy,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...
i really liked this strain for my day time i was able to function and even eat,0.3124,0.1314,0.2614,-0.1815
made me giggly with a very noticeable headkick and a deep relaxation great hybrid,0.6637,-0.7192,-0.1042,0.1504
just hit the bammer with this bud wow i feel great i definitley recommend,0.8915,-0.5320,0.1498,0.0454


## NMF

In [104]:
nmf_model = NMF(3)
doc_topic = nmf_model.fit_transform(doc_word)

In [105]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2","component_3"],
             columns = vectorizer.get_feature_names())
topic_word

,amazing,anxiety,best,body,day,favorite,flavor,head,nice,pain,recommend,relaxed,relaxing,sleep,smell,smoked,smooth,strong,super,sweet,taste
component_1,1.983,0.221,1.347,0.337,0.859,1.160,1.024,0.525,0.000,0.000,0.670,0.577,0.724,0.285,7.458,1.088,1.493,2.399,0.985,3.366,13.107
component_2,0.993,2.886,1.419,3.350,1.899,1.079,0.412,1.859,0.000,10.118,1.091,1.210,1.090,1.734,0.000,0.936,0.307,0.923,0.589,0.230,0.000
component_3,0.000,0.171,0.000,2.529,0.794,0.124,0.958,1.770,11.895,0.000,0.319,0.792,1.390,0.281,0.195,0.402,0.923,0.532,0.460,0.456,0.000


In [106]:
display_topics(nmf_model, vectorizer.get_feature_names(), 5)


Topic  0
taste, smell, sweet, strong, amazing

Topic  1
pain, body, anxiety, day, head

Topic  2
nice, body, head, relaxing, flavor


In [107]:
H = pd.DataFrame(doc_topic.round(5),
             index = strain_1['clean_review'],
             columns = ["component_1","component_2","component_3" ])
H

,component_1,component_2,component_3
clean_review,,,
head body nice flavor or min before full affect clear headed but very relaxed reduced pain productive clear thinking very nice overall feeling not stuck to couch but body fully relaxed after a long week this is new favorite i am ready for bed hrs after a good vape taste with a little addition an hour ago and so is it good nite,0.05052,0.12650,0.19262
love this strain happy calm relaxed but not glued down i can really settle in to a show or game and enjoy either i think this will be a favorite for a long time,0.00551,0.01491,0.00359
are you looking to get shire smacked in the face with a peppy yet tranquil bud if so look no further than this strain with colorful fluffy nugs and beautiful aroma this shit really is the precious smokes great and the flavor is stellar the high gives you a relaxed stroking your gandalf beard in the shire feel,0.00508,0.00947,0.00959
such a smooth smoke instant head high very meoow on the lungs but very potent for pain such a smoke i picked some up at the verdes foundation in abq nm what amazing pre rolls of this im looking forward to the next time they have this in,0.00894,0.09141,0.00533
i now truly understand the beauty of middle earth after smoking this earth blended gift from the fairies i no longer fear the nameless enemy,0.00000,0.00000,0.00000
...,...,...,...
i really liked this strain for my day time i was able to function and even eat,0.00230,0.01260,0.00334
made me giggly with a very noticeable headkick and a deep relaxation great hybrid,0.00000,0.00000,0.00000
just hit the bammer with this bud wow i feel great i definitley recommend,0.00204,0.00728,0.00099


In [108]:
doc_word2=doc_word.copy()
doc_word2 = pd.DataFrame(doc_word2.toarray())

In [109]:
doc_word2['strain_name'] = strain_1["strain_name"]


In [110]:
doc_word2.shape

(165928, 22)

In [111]:
doc_word2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,strain_name
0,0,0,0,2,0,1,1,1,2,1,0,2,0,0,0,0,0,0,0,0,1,gorkle
1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,gorkle
2,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,longbottom-leaf
3,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,longbottom-leaf
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,longbottom-leaf


In [112]:
strain_groupby=doc_word2.groupby(['strain_name']).sum()
strain_groupby

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
strain_name,,,,,,,,,,,,,,,,,,,,,
100-og,20,12,27,20,8,8,8,22,35,25,12,11,8,9,18,17,10,11,14,10,34
13-dawgs,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
22,2,0,2,0,0,0,0,1,0,2,1,1,0,2,0,2,0,1,0,1,1
24k-gold,10,6,10,15,17,9,6,11,26,17,7,11,10,8,20,5,9,14,3,4,26
3-bears-og,0,1,2,3,0,0,0,0,2,2,0,2,1,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zellys-gift,0,2,1,1,1,2,2,3,3,1,1,0,0,0,3,4,1,2,0,0,3
zen,0,1,0,0,4,1,0,0,4,1,2,0,0,1,0,1,0,0,3,1,1
zeta-sage,4,8,4,6,7,6,3,3,4,9,1,2,4,3,7,2,1,2,2,2,10


In [114]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(strain_groupby)


TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [115]:
doc_word_2 = tfidf_transformer.fit_transform(strain_groupby)
doc_word_2

<1859x21 sparse matrix of type '<class 'numpy.float64'>'
	with 25497 stored elements in Compressed Sparse Row format>

In [116]:
lsa = TruncatedSVD(4)
doc_topic = lsa.fit_transform(doc_word_2)
lsa.explained_variance_ratio_

array([0.10010016, 0.09985844, 0.06733635, 0.06367561])

In [117]:
topic_word = pd.DataFrame(lsa.components_.round(4),
             index = ["component_1","component_2","component_3","component_4"],
             columns = vectorizer.get_feature_names())
topic_word

,amazing,anxiety,best,body,day,favorite,flavor,head,nice,pain,recommend,relaxed,relaxing,sleep,smell,smoked,smooth,strong,super,sweet,taste
component_1,0.1855,0.1907,0.2021,0.2636,0.2172,0.1917,0.1539,0.2026,0.3485,0.2949,0.1398,0.1629,0.1956,0.1594,0.2589,0.1820,0.1522,0.1984,0.1500,0.1716,0.3708
component_2,-0.0414,0.1524,0.0350,0.2327,-0.0692,-0.0774,-0.0534,-0.0037,-0.2378,0.7000,0.0109,0.0985,0.1231,0.2806,-0.2740,-0.0424,-0.0104,-0.0442,-0.0003,-0.1684,-0.3854
component_3,-0.1276,0.0076,0.0091,0.1973,0.1606,-0.0051,-0.0311,0.2845,0.6641,-0.2037,0.0018,0.0324,-0.0541,-0.0770,-0.2519,0.0869,-0.0314,0.0029,-0.0545,-0.1614,-0.4935
component_4,0.1015,0.2450,0.4171,-0.2155,0.3169,0.2513,0.0733,0.0488,-0.3443,-0.0622,0.1342,-0.0633,-0.3106,-0.1778,-0.0750,0.4249,-0.0030,0.0484,0.1307,-0.0686,-0.2349


In [118]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [119]:
display_topics(lsa, vectorizer.get_feature_names(), 10)


Topic  0
taste, nice, pain, body, smell, day, head, best, strong, relaxing

Topic  1
pain, sleep, body, anxiety, relaxing, relaxed, best, recommend, super, head

Topic  2
nice, head, body, day, smoked, relaxed, best, anxiety, strong, recommend

Topic  3
smoked, best, day, favorite, anxiety, recommend, super, amazing, flavor, head


In [121]:
from sklearn.preprocessing import StandardScaler

In [122]:
sscale = StandardScaler()

In [123]:
pd.DataFrame(doc_topic)

,0,1,2,3
0,0.962204,-0.001607,0.036824,0.051035
1,0.390855,-0.160674,-0.361671,-0.376555
2,0.633631,0.272172,-0.176377,0.243858
3,0.972277,-0.048209,0.025376,-0.086915
4,0.712177,0.198403,0.013882,-0.229292
...,...,...,...,...
1854,0.800663,-0.212644,0.121917,0.246720
1855,0.621903,-0.083085,0.279741,0.171775
1856,0.923798,0.113929,-0.160565,0.084519
1857,0.789155,0.062135,0.156434,-0.195053


In [124]:
strain_cosine=pd.DataFrame(cosine_similarity(doc_topic),columns=strain_groupby.index.values,index=strain_groupby.index.values)

In [141]:
strain_cosine['blueberry'].sort_values(ascending=False)

blueberry                 1.000000
swiss-tsunami             0.999361
cali-kush                 0.999180
ice-cream                 0.999078
golden-goat               0.999009
                            ...   
godzilla-blood            0.000000
the-original-lemonnade    0.000000
space-poison              0.000000
citral-skunk              0.000000
pinewarp                  0.000000
Name: blueberry, Length: 1859, dtype: float64

In [142]:
blueberry_recommend=pd.DataFrame(strain_cosine['blueberry'].sort_values(ascending=False))

In [128]:
gelato_recommend=pd.DataFrame(strain_cosine['gelato'].sort_values(ascending=False))

In [143]:
blueberry_recommend.head(11)

,blueberry
blueberry,1.000000
swiss-tsunami,0.999361
cali-kush,0.999180
ice-cream,0.999078
golden-goat,0.999009
alien-og,0.998996
honey-bananas,0.998944
ayahuasca-purple,0.998565
sequoia-strawberry,0.998543
berry-bomb,0.998518


In [126]:
lsa.explained_variance_ratio_

array([0.10010016, 0.09985844, 0.06733635, 0.06367561])

## LDA

In [ ]:
strain_sparse=csc_matrix(strain_groupby)

In [ ]:
strain_sparse.shape

In [ ]:
strain_sparse

In [ ]:
corpus = matutils.Sparse2Corpus(strain_sparse)

In [ ]:
lda = models.LdaModel(corpus=corpus, num_topics=3, id2word=id2word, passes=5)

In [ ]:
len(id2word)

In [ ]:
lda = models.LdaModel(corpus=corpus, num_topics=3,passes=5)

## LDA with Sklearn

In [ ]:
lda_model = LatentDirichletAllocation(n_components=4,max_iter=100)
lda_doc_topics=lda_model.fit_transform(strain_groupby)

In [ ]:
lda_model.components_[0]

In [ ]:
len(lda_model.components_)

In [ ]:
topic_word_lda = pd.DataFrame(lda_model.components_.round(4),
             index = ["component_1","component_2","component_3","component_4"],
             columns = vectorizer.get_feature_names())
topic_word_lda

In [ ]:
display_topics(lda_model, vectorizer.get_feature_names(), 10)

In [ ]:
topic